In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
""" Example for computing the RIR between several sources and receivers in GPU.
"""

import os
import numpy as np
from math import ceil, cos, sin
import math
from scipy.io import wavfile
from multiprocessing import Pool
import multiprocessing
import rir_generator as rir


fs = 16000

def main(key, room_sz, T60, save_dir):
    # import gpuRIR
    # gpuRIR.activateMixedPrecision(False)
    # gpuRIR.activateLUT(True)

    deg_60 = math.pi/3
    mic_1 = np.array([2.0, 1.5, 0.15])
    d = 0.08
    mic_2 = mic_1 + np.array([d*cos(deg_60), d*sin(deg_60), 0])
    mic_array = np.array([
                [mic_1[0], mic_1[1], mic_1[2]], 
                [mic_2[0], mic_2[1], mic_2[2]], 
                ])
        
    for idx in range(10):
        pos_src = mic_1
        while np.linalg.norm(pos_src[0:2] - mic_1[0:2]) < 3.0:
            x = np.random.uniform(low=0.3,high=room_sz[0]-0.3)
            y = np.random.uniform(low=0.3,high=room_sz[1]-0.3)
            z = np.random.uniform(low=1.0,high=1.5)
            pos_src = np.array([x,y,z]) # Positions of the sources ([m]
            
        # mic_pattern = "omni" # Receiver polar pattern
        # orV_rcv=None # None for omni

        # abs_weights = [0.9]*5+[0.5] # Absortion coefficient ratios of the walls
        # att_diff = 25.0    # !Attenuation when start using the diffuse reverberation model [dB]
        # att_max = 60.0 # Attenuation at the end of the simulation [dB]

        # beta = gpuRIR.beta_SabineEstimation(room_sz, T60, abs_weights=abs_weights) # Reflection coefficients
        # Tdiff= gpuRIR.att2t_SabineEstimator(att_diff, T60) # Time to start the diffuse reverberation model [s]
        # Tmax = gpuRIR.att2t_SabineEstimator(att_max, T60)     # Time to stop the simulation [s]
        # nb_img = gpuRIR.t2n( Tdiff, room_sz )    # Number of image sources in each dimension
        
        # # RIR generation
        # RIRs = gpuRIR.simulateRIR(room_sz, beta, np.expand_dims(pos_src,axis=0), mic_array, nb_img, Tmax, fs, Tdiff=Tdiff, orV_rcv=orV_rcv, mic_pattern=mic_pattern)
        RIRs = rir.generate(
            c=340,                  # Sound velocity (m/s)
            fs=fs,                  # Sample frequency (samples/s)
            r=mic_array,
            s=pos_src,          # Source position [x y z] (m)
            L=room_sz,            # Room dimensions [x y z] (m)
            reverberation_time=T60, # Reverberation time (s)
            nsample=int(T60*fs),           # Number of output samples
        )
        # print(RIRs.shape); raise()
        sub_dir = '/RIR_sample_room_'+key+'/T60_'+str(T60)
        file_name = '/location_'+str(round(x,2))+'_'+str(round(y,2))+'_'+str(round(z,2))+'.wav'
        os.makedirs(save_dir+sub_dir, exist_ok=True)
        wavfile.write(save_dir+sub_dir+file_name, fs, RIRs)

    print("Generating RIR filters for Room = {}, RT = {} done.".format(key, T60))

def chunkify(lst, n):
    return [lst[i::n] for i in range(n)]

if __name__ == "__main__":
    rt_list = [0.25, 0.3, 0.35, 0.4]
    rooms = {
             'medium_0':[8, 6, 3],
             }
    root_dir = "./RIR_filters"
    dict_list = [(key,room_sz,T60, root_dir) 
                 for T60 in rt_list
                 for key, room_sz in rooms.items()]
    for room in rooms:
        for T60 in rt_list:
            main(room, rooms[room], T60, root_dir)
    # num_chunks = 10  # 프로세스 수를 CPU 코어 수로 설정
    # chunked_dict_list = chunkify(dict_list, num_chunks)

    # with Pool(num_chunks) as p:
    #     for chunk in chunked_dict_list:
    #         p.starmap(main, chunk)

Generating RIR filters for Room = medium_0, RT = 0.25 done.
Generating RIR filters for Room = medium_0, RT = 0.3 done.
Generating RIR filters for Room = medium_0, RT = 0.35 done.
Generating RIR filters for Room = medium_0, RT = 0.4 done.


In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
""" Example for computing the RIR between several sources and receivers in GPU.
"""

import os
import numpy as np
from math import ceil, cos, sin
import math
from scipy.io import wavfile
from multiprocessing import Pool
import multiprocessing
import rir_generator as rir


fs = 16000

def main(key, room_sz, T60, save_dir):
    deg_60 = math.pi/3
    mic_1 = np.array([2, 1.5, 0.15])
    d = 0.08
    mic_2 = mic_1 + np.array([d*cos(deg_60), d*sin(deg_60), 0])
    mic_array = np.array([
                [mic_1[0], mic_1[1], mic_1[2]], 
                [mic_2[0], mic_2[1], mic_2[2]], 
                ])
        
    d_echo = 0.3
    pos_src = mic_1 + np.array([d_echo*cos(deg_60*2.5), d_echo*sin(deg_60*2.5), 0])
    x, y, z = pos_src
    mic_pattern = "omni" # Receiver polar pattern
    # orV_rcv=None # None for omni

    # abs_weights = [0.9]*5+[0.5] # Absortion coefficient ratios of the walls
    # att_diff = 25.0    # !Attenuation when start using the diffuse reverberation model [dB]
    # att_max = 60.0 # Attenuation at the end of the simulation [dB]

    # beta = gpuRIR.beta_SabineEstimation(room_sz, T60, abs_weights=abs_weights) # Reflection coefficients
    # Tdiff= gpuRIR.att2t_SabineEstimator(att_diff, T60) # Time to start the diffuse reverberation model [s]
    # Tmax = gpuRIR.att2t_SabineEstimator(att_max, T60)     # Time to stop the simulation [s]
    # nb_img = gpuRIR.t2n( Tdiff, room_sz )    # Number of image sources in each dimension
    
    # # RIR generation
    # RIRs = gpuRIR.simulateRIR(room_sz, beta, np.expand_dims(pos_src,axis=0), mic_array, nb_img, Tmax, fs, Tdiff=Tdiff, orV_rcv=orV_rcv, mic_pattern=mic_pattern)

    RIRs = rir.generate(
        c=340,                  # Sound velocity (m/s)
        fs=fs,                  # Sample frequency (samples/s)
        r=mic_array,
        s=pos_src,          # Source position [x y z] (m)
        L=room_sz,            # Room dimensions [x y z] (m)
        reverberation_time=T60, # Reverberation time (s)
        nsample=int(T60*fs),           # Number of output samples
    )

    sub_dir = '/RIR_sample_room_'+key+'/T60_'+str(T60)
    file_name = '/echo_'+str(round(x,2))+'_'+str(round(y,2))+'_'+str(round(z,2))+'.wav'
    os.makedirs(save_dir+sub_dir, exist_ok=True)
    wavfile.write(save_dir+sub_dir+file_name, fs, RIRs)
    
    print("Generating RIR filters for Room = {}, RT = {} done.".format(key, T60))
    
def chunkify(lst, n):
    return [lst[i::n] for i in range(n)]

if __name__ == "__main__":
    rt_list = [0.25, 0.3, 0.35, 0.4]
    rooms = {
             'medium_0':[8, 6, 3],
             }
    root_dir = "./RIR_filters"
    dict_list = [(key,room_sz,T60, root_dir) 
                 for T60 in rt_list
                 for key, room_sz in rooms.items()]
    
    for room in rooms:
        for T60 in rt_list:
            main(room, rooms[room], T60, root_dir)
    

Generating RIR filters for Room = medium_0, RT = 0.25 done.
Generating RIR filters for Room = medium_0, RT = 0.3 done.
Generating RIR filters for Room = medium_0, RT = 0.35 done.
Generating RIR filters for Room = medium_0, RT = 0.4 done.
